In [1]:
include("2_4.jl")
#inputfile = ARGS[1]
inputfilepath = "data_2-4"
inputfilename = "Data_ppb_tournee_d5m10p10.2.txt"
nb_dem, nb_prod, nb_mag, nb_noeuds, matrice_S, matrice_Q, matrice_R = read_data_24(inputfilepath, inputfilename)
println(nb_dem)

5


In [2]:
nb_dem, nb_prod, nb_mag, nb_noeuds, S, Q, R = read_data_24(inputfilepath, inputfilename)
println(S[nb_mag][nb_prod]) #stock du dernier produit dans le dernier magasin
println(R[nb_mag][nb_mag+1]) #distance entre le dernier magasin et le premier client
println(nb_noeuds)

0.0
707.0
15


In [3]:
#importer les packages utiles, le manager de package Pkg etant un package
import Pkg; Pkg.add("Cbc")
Pkg.add("JuMP")

   Updating registry at `C:\Users\thoma\.julia\registries\General`
  Resolving package versions...
No Changes to `C:\Users\thoma\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\thoma\.julia\environments\v1.5\Manifest.toml`
  Resolving package versions...
No Changes to `C:\Users\thoma\.julia\environments\v1.5\Project.toml`
No Changes to `C:\Users\thoma\.julia\environments\v1.5\Manifest.toml`


In [3]:
using Cbc
using JuMP

somme_stocks = sum(sum(matrice_S))


# set optimizer
model = Model(Cbc.Optimizer)

# variables
@variable(model,arc_emprunte[1:nb_noeuds,1:nb_noeuds,1:nb_mag],Bin)
@variable(model, binaire[1:nb_mag,1:nb_dem], Bin)
@variable(model, quantites[1:nb_mag,1:nb_dem,1:nb_prod] >= 0, Int)
@variable(model, 0<=ordre[1:nb_mag,1:nb_noeuds],Int)

# fonction objectif
@objective(model, Min, sum(sum(sum(arc_emprunte[i,j,m]*R[i][j] for i in 1:nb_noeuds) for j in 1:nb_noeuds) for m in 1:nb_mag))

# Contraintes

for p in 1:nb_prod
    for d in 1:nb_dem
        # Contrainte 3. Chaque commande doit être satisfaite en totalité
        @constraint(model, sum(quantites[m,d,p] for m in 1:nb_mag) == matrice_Q[d][p]) 
    end
    for m in 1:nb_mag
        # Contrainte 5. Quantites vendues <= Stocks des magasins
        @constraint(model, sum(quantites[m,d,p] for d in 1:nb_dem) <= matrice_S[m][p])
    end
end

for m in 1:nb_mag
    # Contrainte 2.
    # Le livreur commence sa tournée au magasin
    @constraint(model, sum(arc_emprunte[m,j,m] for j in nb_mag+1:nb_noeuds) == 1 )
    # Le livreur finit sa tournée au magasin
    @constraint(model, sum(arc_emprunte[i,m,m] for i in nb_mag+1:nb_noeuds) == 1 )
    # Le livreur d'un magasin ne se déplace pas jusqu'à un autre magasin
    @constraint(model, sum(sum(arc_emprunte[i,j,m] for j in 1:m-1) for i in 1:nb_noeuds ) == 0)
    @constraint(model, sum(sum(arc_emprunte[i,j,m] for j in m+1:nb_mag) for i in 1:nb_noeuds ) == 0)
    # Le livreur d'un magasin ne se déplace pas à partir d'un autre magasin
    @constraint(model, sum(sum(arc_emprunte[i,j,m] for i in 1:m-1) for j in 1:nb_noeuds ) == 0)
    @constraint(model, sum(sum(arc_emprunte[i,j,m] for i in m+1:nb_mag) for j in 1:nb_noeuds ) == 0)
    for i in 1:nb_noeuds
        # Le livreur ne fait pas du sur place
        @constraint(model, arc_emprunte[i,i,m] == 0)
    end
    for i in (nb_mag+1):nb_noeuds
        for j in 1:nb_noeuds
            # Le livreur doit passer par un noeud pour pouvoir en partir => ordre
            @constraint(model, 1+ordre[m,i]<=ordre[m,j] + nb_noeuds*(1-arc_emprunte[i,j,m]))
        end
    end
    for d in 1:nb_dem
        # Le livreur part de chaque noeud client et seulement les noeuds clients
        @constraint(model, somme_stocks*sum(arc_emprunte[nb_mag+d,j,m] for j in 1:nb_noeuds)>= sum(quantites[m,d,p] for p in 1:nb_prod))
        @constraint(model, sum(arc_emprunte[nb_mag+d,j,m] for j in 1:nb_noeuds)<= sum(quantites[m,d,p] for p in 1:nb_prod))
        # Le livreur arrive vers chaque noeud client et seulement vers les noeuds clients
        @constraint(model, somme_stocks*sum(arc_emprunte[i,nb_mag+d,m] for i in 1:nb_noeuds)>= sum(quantites[m,d,p] for p in 1:nb_prod))
        @constraint(model, sum(arc_emprunte[i,nb_mag+d,m] for i in 1:nb_noeuds)<= sum(quantites[m,d,p] for p in 1:nb_prod))

    end
end

# run optimization
optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 4888.79 - 0.00 seconds
Cgl0002I 1950 variables fixed
Cgl0003I 0 fixed, 0 tightened bounds, 47 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 47 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 46 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 41 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 2 strengthened rows, 0 substitutions
Cgl0004I processed model has 505 rows, 446 columns (446 integer (366 of which binary)) and 2531 elements
Cbc0031I 107 added rows had average density of 17.925234
Cbc0013I At root node, 107 cuts changed objective from 7427.0487 to 10382.685 in 100 p

In [4]:
#print solution
println("Solution obtenue:")
println("distance totale = $(objective_value(model))")
for m in 1:nb_mag
    dm = sum(sum(value(arc_emprunte[i,j,m])*R[i][j] for i in 1:nb_noeuds) for j in 1:nb_noeuds)
    println("\t distance parcourue par le livreur du magasin $m = $dm")
end

for m in 1:nb_mag
    println("")
    println("Livreur du magasin $m :")
    for i in 1:nb_noeuds
        for j in 1:nb_noeuds
            if (value(arc_emprunte[i,j,m])>0.5)
                print("\t ($i,$j)")
            end
        end
    end
    println("")
end

for m in 1:nb_mag
    println("")
    println("Arcs empruntés magasin $m :")
    for i in 1:nb_noeuds
        println("")
        for j in 1:nb_noeuds
            print("\t $(value(arc_emprunte[i,j,m]))")
        end
    end
end

for m in 1:nb_mag
    println("")
    println("Quantités vendues $m :")
    for dm in 1:nb_dem
        println("")
        for p in 1:nb_prod
            print("\t $(value(quantites[m,dm,p]))")
        end
    end
end

for m in 1:nb_mag
    println("")
    println("Ordre magasin $m :")
    println("")
    for i in 1:nb_noeuds
        print("\t $(value(ordre[m,i]))")
    end
end



Solution obtenue:
distance totale = 11001.0
	 distance parcourue par le livreur du magasin 1 = 468.0
	 distance parcourue par le livreur du magasin 2 = 704.0
	 distance parcourue par le livreur du magasin 3 = 1446.0
	 distance parcourue par le livreur du magasin 4 = 940.0
	 distance parcourue par le livreur du magasin 5 = 1510.0
	 distance parcourue par le livreur du magasin 6 = 2142.0
	 distance parcourue par le livreur du magasin 7 = 856.0
	 distance parcourue par le livreur du magasin 8 = 368.0
	 distance parcourue par le livreur du magasin 9 = 2009.0
	 distance parcourue par le livreur du magasin 10 = 558.0

Livreur du magasin 1 :
	 (1,14)	 (14,1)

Livreur du magasin 2 :
	 (2,15)	 (14,2)	 (15,14)

Livreur du magasin 3 :
	 (3,11)	 (11,3)

Livreur du magasin 4 :
	 (4,12)	 (12,13)	 (13,4)

Livreur du magasin 5 :
	 (5,15)	 (15,5)

Livreur du magasin 6 :
	 (6,14)	 (11,6)	 (14,11)

Livreur du magasin 7 :
	 (7,14)	 (14,7)

Livreur du magasin 8 :
	 (8,11)	 (11,8)

Livreur du magasin 9 :
	 

	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0
	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0
	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0
	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0
	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 1.0	 0.0	 0.0
	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0
	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0
	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 1.0	 0.0	 0.0	 0.0	 0.0	 0.0
	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0
	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0
Quantités vendues 1 :

	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0
	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0
	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0	 0.0
	 2.0	 0.0	 0.0	 0.0	 0.0	 1.0	 0.0	 1.0	 1.0	 0.0
	 0.0	 0.0	 0